# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-12 07:02:27] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-12 07:02:27] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-12 07:02:27] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-12 07:02:30] INFO server_args.py:1643: Attention backend not specified. Use fa3 backend by default.


[2026-01-12 07:02:30] INFO server_args.py:2542: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=76.93 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=72 avail_mem=76.80 GB):  20%|██        | 4/20 [00:00<00:01, 10.04it/s]

Capturing batches (bs=24 avail_mem=76.77 GB):  55%|█████▌    | 11/20 [00:00<00:00, 20.49it/s]

Capturing batches (bs=1 avail_mem=76.74 GB):  95%|█████████▌| 19/20 [00:00<00:00, 25.34it/s]

Capturing batches (bs=1 avail_mem=76.74 GB): 100%|██████████| 20/20 [00:01<00:00, 19.57it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Paul and I'm a software engineer. I have a B.S. in Computer Science from UC Berkeley and have worked on a wide variety of projects, but the most popular is the OpenCV project which I'm a part of. It's a software library that can take images and turn them into machine learning models, so it's really fun to code. I'm really good at trying to solve problems, and programming is a great way to do that. I like to use photography, I like to shoot photos, and then I like to turn them into computer models, so that's part of my hobby as well. How is your day
Prompt: The president of the United States is
Generated text:  the head of the executive branch of the federal government, as appointed by the President of the United States. The President appoints and unapoints members of Congress, and his or her immediate family, such as his or her children, is eligible for a term of office. The U.S. Constitution provides that each president can only serve a maxim

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [Age] year old [Gender] [Occupation]. I'm currently [Current Location] and I'm here to [Your First Goal/Challenge]. I'm excited to meet you and learn more about you. What's your name? What's your occupation? What's your current location? What's your goal or challenge? I'm looking forward to meeting you. [Your Name] [Your Occupation] [Your Current Location] [Your Goal/Challenge] [Your Name] [Your Occupation] [Your Current Location] [Your Goal/Challenge] [Your Name] [Your Occupation] [

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as "La Ville-Marie" or "La Ville de Paris" and is the largest city in France by population. It is located on the Seine River and is the seat of the French government and the country's cultural, political, and economic center. Paris is known for its rich history, art, and architecture, and is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. The city is also known for its fashion industry, with many famous fashion designers and boutiques located in the city. Paris is a popular tourist destination and is a major

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction. Here are some of the most likely trends:

1. Increased focus on ethical AI: As more people become aware of the potential risks of AI, there is a growing emphasis on developing AI that is designed to be ethical and responsible. This could include developing AI that is designed to minimize harm to individuals or society as a whole, or that is designed to be transparent and accountable.

2. Greater use of AI in healthcare: AI is already being used in healthcare to help diagnose and treat diseases, and there is a growing expectation that it will be used



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a 23-year-old software engineer. I have a passion for technology and love to challenge myself to solve complex problems. I'm always looking for new ways to improve my skills and grow as a person. I'm a friendly, outgoing person who enjoys meeting new people and trying out new projects. I'm excited to learn and grow with every new challenge that I face. If you need any help or advice, please don't hesitate to reach out. I look forward to meeting you! 

-Please respond in a neutral tone and keep the conversation light and casual. Use a conversational, friendly, and

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the most populous city in the country.

Here's a concise factual statement about France's capital city:

Paris is the most populous city in France, with an estimated population of 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 ____

 and

 I

 am

...

 repetitive

 tasks

,

 freeing

 up

In [6]:
llm.shutdown()